G - generative
D - Discriminator

G neural network, takes random noise signal and output some image, theif trying to steal
D another neural network, rivaling generator, police trying to detect, understands what the generator is trying to do

### APPLICATIONS

generating images<br>
image modification<br>
super resolution<br>
assisting artists<br>
photo-realistic images<br>
speech recognition<br>
face aging

image generation is the main application

## import libraries

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

In [2]:
batchsize = 64
imagesize = 64

In [3]:
transform = transforms.Compose([transforms.Scale(imagesize),
                                transforms.ToTensor(),
                                transforms.Normalize(
                                    (0.5, 0.5, 0.5),
                                    (0.5, 0.5, 0.5)
                                ),
                               ]) # We create a list of transformations (scaling, tensor conversion, normalization) to apply to the input images.

/home/wajeeh-machine/anaconda3/envs/mytf2/lib/python3.7/site-packages/torchvision/transforms/transforms.py:279: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


### load dataset

In [4]:
! ls ../datasets/gan_cifar10/data

cifar-10-batches-py


In [5]:
dataset = dset.CIFAR10(root = '../datasets/gan_cifar10/data',
                      download = True, transform = transform)

Files already downloaded and verified


dataloader gives us data batch to batch

In [6]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size = batchsize,
                                        shuffle=True, num_workers=2)

weights will take neural network and initialize weights for both networks

In [7]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

defining generator, its neural network and forward function

In [8]:
class G(nn.Module):
    
    def __init__(self):
        ''' initialize the generator neural network'''
        super(G, self).__init__()
        
        self.main = nn.Sequential( # We create a meta module of a neural network that will contain a sequence of modules (convolutions, full connections, etc.).
            nn.ConvTranspose2d(100, 512, 4, 1, 0, bias = False), # We start with an inversed convolution.
            nn.BatchNorm2d(512), # We normalize all the features along the dimension of the batch.
            nn.ReLU(True), # We apply a ReLU rectification to break the linearity.
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias = False), # We add another inversed convolution.
            nn.BatchNorm2d(256), # We normalize again.
            nn.ReLU(True), # We apply another ReLU.
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias = False), # We add another inversed convolution.
            nn.BatchNorm2d(128), # We normalize again.
            nn.ReLU(True), # We apply another ReLU.
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias = False), # We add another inversed convolution.
            nn.BatchNorm2d(64), # We normalize again.
            nn.ReLU(True), # We apply another ReLU.
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias = False), # We add another inversed convolution.
            nn.Tanh() # We apply a Tanh rectification to break the linearity and stay between -1 and +1.
        )
        
    def forward(self, input):
        ''' it takes in the network, forward propagate and returns 
        the generated image'''
        output = self.main(input) # forward propagate through whole nw
        return output # return o/p containint generated image

##### creating Generator object 

In [9]:
netG = G() # create generator object
netG.apply(weights_init) # initialize all the weights of its neural network

G(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

## defining discriminator, its neural network and forward function

In [10]:
class D(nn.Module):
    
    def __init__(self):
        '''initialize discriminator neural network'''
        super(D, self).__init__()
        
        self.main = nn.Sequential(
            
        nn.Conv2d(3, 64, 4, 2, 1, bias = False),
        nn.LeakyReLU(0.2, inplace = True),
        nn.Conv2d(64, 128, 4, 2, 1, bias = False),
        nn.BatchNorm2d(128),
        nn.LeakyReLU(0.2, inplace = True),
        nn.Conv2d(128, 256, 4, 2, 1, bias = True),
        nn.BatchNorm2d(256),
        nn.LeakyReLU(0.2, inplace = True),
        nn.Conv2d(256, 512, 4, 2, 1, bias = False),
        nn.BatchNorm2d(512),
        nn.LeakyReLU(0.2, inplace = True),
        nn.Conv2d(512, 1, 4, 1, 0, bias = False),
        nn.Sigmoid()
            
        )
        
    def forward(self, input):
        output = self.main(input) # forward propagate through whole nw
        return output.view(-1) # return output value b/w 0 or 1

##### creating discriminator object

torch.module.apply(fn) Applies fn recursively to every submodule (as returned by .children()) as well as self. Typical use includes initializing the parameters of a model

In [11]:
netD = D() # create discriminator object
netD.apply(weights_init) # initialize all the weightsof its neural network

D(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)

### training DCGANs 

In [12]:
criterion = nn.BCELoss() # Criterion will measure the error between prediction and target

In [13]:
# optimizer for the discriminator
optimizerD = optim.Adam(netD.parameters(), lr= 0.0002,
                        betas = (0.5, 0.999))

In [14]:
# optimizer for the generator
optimizerG = optim.Adam(netG.parameters(), lr = 0.0002,
                       betas = (0.5, 0.999))

In [15]:
total_epochs = 10

In [16]:
for epoch in range(total_epochs):
    for i, data in enumerate(dataloader, 0): # iterate over image of dataset
        # 1st step: Updating weights of neural network of discriminator
        netD.zero_grad() # we initialize to 0 the gradients of the discriminator w.r.t its weights
        # Training discriminator with real image of the dataset which will be used to train the discriminator
        real, _ = data # we get a real image of dataset which will train the discriminator
        input = Variable(real) # we wrap it in a Variable
        target = Variable(torch.ones(input.size()[0])) # we get the target
        output = netD(input) # we forward propagate this real image into nn of discriminator to get prediction between 1 or 0
        errD_real = criterion(output, target) # we compute loss b/w predictions op and target equal to 1
        
        # Training the discriminator with a fake image generated by generator
        noise = Variable(torch.randn(input.size()[0], 100, 1, 1)) # made a random input vector (noise) of the generator
        fake = netG(noise) # we forward propagate this random input vector into nn of generator to get some fake generated image
        target = Variable(torch.zeros(input.size()[0])) # we get the target
        output = netD(fake.detach()) # we forward propagate the fake generated image into the network of the discriminator to get the prediction between 0 or 1
        errD_fake = criterion(output, target) # we compute thhe loss b/w prediction (output) and the target (equal to 0)
        
        #BCK PROPAGATING THE TOTAL ERROR
        errD = errD_real + errD_fake # we compute total error of discriminator
        errD.backward() # we backpropagate the loss error by computing the gradients of total respect to the weights of the discriminator
        optimizerD.step() #we apply the optimizer to update the weights according to how much they are responsible for the loss of the discriminator
        
        # 2nd step: updating the weights of the neural network of the generator
        netG.zero_grad() # initialize to the 0 the gradients of the generator with respect to the weights
        target = Variable(torch.ones(input.size()[0])) # we get the target
        output = netD(fake) # we forward propagate the fake generated images into the neural of the discriminator to get the prediction (val bw 0 and 1)
        errG = criterion(output, target) # we compute the loss error by computing the gradients of the total with respoect to the weights of the generator
        optimizerG.step() # we apply optimizer to update the weights according to how much they are responsible for the loss error of the generator
        
        # 3rd Step: printing the losses and saving the real image and the generated image of the minibatch every 100 steps
        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f' % (epoch, 25, i, len(dataloader), errD.item(), errG.item())) # We print les losses of the discriminator (Loss_D) and the generator (Loss_G).
        
        if i % 100 == 0:
            vutils.save_image(real, '%s/real_samples.png' % "../datasets/gan_cifar10/results/", normalize = True) # We save the real images of the minibatch.
            fake = netG(noise) #We get our fake generated images.
            vutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png' % ("../datasets/gan_cifar10/results/", epoch), normalize = True) # We also save the fake generated images of the minibatch.
        

[0/25][0/782] Loss_D: 1.5275 Loss_G: 4.8386
[0/25][1/782] Loss_D: 0.3197 Loss_G: 4.4910
[0/25][2/782] Loss_D: 0.0574 Loss_G: 5.1611
[0/25][3/782] Loss_D: 0.0340 Loss_G: 5.6326
[0/25][4/782] Loss_D: 0.0118 Loss_G: 6.3019
[0/25][5/782] Loss_D: 0.0086 Loss_G: 6.6130
[0/25][6/782] Loss_D: 0.0065 Loss_G: 6.9921
[0/25][7/782] Loss_D: 0.0054 Loss_G: 6.8979
[0/25][8/782] Loss_D: 0.0039 Loss_G: 7.3516
[0/25][9/782] Loss_D: 0.0032 Loss_G: 7.4955
[0/25][10/782] Loss_D: 0.0034 Loss_G: 7.6257
[0/25][11/782] Loss_D: 0.0033 Loss_G: 7.6747
[0/25][12/782] Loss_D: 0.0034 Loss_G: 7.8878
[0/25][13/782] Loss_D: 0.0031 Loss_G: 7.9162
[0/25][14/782] Loss_D: 0.0025 Loss_G: 7.8785
[0/25][15/782] Loss_D: 0.0025 Loss_G: 7.7412
[0/25][16/782] Loss_D: 0.0025 Loss_G: 7.8952
[0/25][17/782] Loss_D: 0.0024 Loss_G: 8.0970
[0/25][18/782] Loss_D: 0.0025 Loss_G: 8.1430
[0/25][19/782] Loss_D: 0.0029 Loss_G: 7.8425
[0/25][20/782] Loss_D: 0.0021 Loss_G: 7.9945
[0/25][21/782] Loss_D: 0.0020 Loss_G: 7.9775
[0/25][22/782] Loss_

KeyboardInterrupt: 

Traceback (most recent call last):
  File "/home/wajeeh-machine/anaconda3/envs/mytf2/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/wajeeh-machine/anaconda3/envs/mytf2/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/wajeeh-machine/anaconda3/envs/mytf2/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/wajeeh-machine/anaconda3/envs/mytf2/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
